In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload

WANDB_enable = False
creds_path_ar = ["../credentials.ini","credentials.colab.ini"]
root_path = ""
data_path = ""

for creds_path in creds_path_ar:
  if path.exists(creds_path):
      config_parser = configparser.ConfigParser()
      config_parser.read(creds_path)
      root_path = config_parser['MAIN']["PATH_ROOT"]
      data_path = config_parser['MAIN']["PATH_DATA"]
      WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
      ENV = config_parser['MAIN']["ENV"]
      break

if ENV=="COLAB" and WANDB_enable:
  !pip install --upgrade wandb

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
cd {root_path}

/content/gdrive/My Drive/WORK/ML/develop/SOTA-NLP/SOTA


In [8]:
import pandas as pd   
import re
import numpy as np
import matplotlib.pyplot as plt
import os   
import scipy
from nltk.corpus import stopwords
from gensim.models import KeyedVectors

from nltk import pos_tag, word_tokenize
import nltk

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
df = pd.read_csv(data_path+"/semantic_similarity_ds/Paraphrase.csv")

In [10]:
df['bleu_allwords'] = 0
df['bleu_withoutstop'] = 0


def preprocess(raw_text,stopwords_remove=True):
    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", str(raw_text))
    # convert to lower case and split 
    words = letters_only_text.lower().split()
    #print(words)
    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    return cleaned_words if stopwords_remove else words

def BLEU1score(s1, s2,stopwords_remove=True):

    hypothesis = [word for word in preprocess(s1,stopwords_remove)]
    reference = [word for word in preprocess(s2,stopwords_remove)]
    #print(hypothesis,",",reference)
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights=(1, 0, 0, 0))
    return BLEUscore


In [11]:
scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=False)
    df.loc[i,'bleu_allwords']=dis

scores = []
for i in range(df.shape[0]):
    s1 = str(df['text_1'][i])
    s2 = str(df['text_2'][i])
    #print(s1,",",s2)
    dis = BLEU1score(s1,s2,stopwords_remove=True)
    df.loc[i,'bleu_withoutstop']=dis

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [12]:
df

,text_1,text_2,label,bleu_allwords,bleu_withoutstop
0,take measures in order to,take additional measures to,4.000,0.600000,0.666667
1,was seeking to,will strive to,3.000,0.333333,0.000000
2,canada aims to,canada works to,2.333,0.666667,0.500000
3,refugees and,refugee population,2.667,0.000000,0.000000
4,measures and,"instruments , and",3.000,0.500000,0.000000
...,...,...,...,...,...
993,unless specified otherwise,unless specifically stated otherwise,4.667,0.477688,0.477688
994,its efforts to combat,the area of combating,3.667,0.000000,0.000000
995,considered by the,give consideration to the,4.333,0.238844,0.000000
996,an enlarged security,an expanded security,4.000,0.666667,0.500000
